# 利用直接消元法以及广义消元法求解等式约束的二次规划问题

\begin{equation*}
\begin{array}{cl}
\text{minimize} & q(x) = x_1^2 + x_2^2 + 2x_3^2 + 5x_1x_2 - 3x_2 - 7x_3, \\
\text{subject to} & x_1 + x_2 + x_3 = 1, \\
& x_1 - 2x_2 - 3x_3 = -2,
\end{array}
\end{equation*}


In [ ]:
import sympy as sp

In [ ]:
x1, x2, x3 = sp.symbols("x1, x2, x3")
qx = x1**2 + x2**2 + 2 * x3**2 + 5 * x1 * x2 - 3 * x2 - 7 * x3

In [ ]:
G = sp.Matrix(3, 3, [2, 5, 0, 5, 2, 0, 0, 0, 4])
d = sp.Matrix(3, 1, [0, -3, -7])
A = sp.Matrix(3, 2, [1, 1, 1, -2, 1, -3])
b = sp.Matrix(2, 1, [1, -2])

## 直接消元法

In [ ]:
psi = sp.simplify(
    qx.subs([(x1, sp.Rational(1, 3) * x3), (x2, 1 - sp.Rational(4, 3) * x3)])
)
psi

In [ ]:
# Hessian

sp.diff(psi, (x3, 2))

In [ ]:
x3_star = list(sp.roots(sp.diff(psi, (x3, 1)), x3))[0]
x3_star

In [ ]:
A1 = A[:2, :]
A2 = A[2:, :]

[x1_star, x2_star] = A1.T.inv() @ (b - A2.T @ sp.Matrix([x3_star]))
x1_star, x2_star

In [ ]:
g = -G @ sp.Matrix(3, 1, [sp.Rational(2, 5), -sp.Rational(3, 5), sp.Rational(6, 5)]) - d
g

In [ ]:
lambda_star = sp.Matrix(2, 1, list(sp.linsolve((A, g)))[0])
lambda_star

## 广义消元法

In [ ]:
q, r = A.QRdecomposition()

In [ ]:
q

In [ ]:
r

In [ ]:
Y = q @ r.T.inv()
Y

In [ ]:
sp.linsolve((q.T, sp.zeros(2, 1))) == sp.linsolve((A.T, sp.zeros(2, 1)))

In [ ]:
Z = sp.Matrix(list(sp.linsolve((A.T, sp.zeros(2, 1)))))
Z = Z.subs("tau0", 1).T
Z = Z / Z.norm()
Z

In [ ]:
# Hessian

Z.T @ G @ Z

In [ ]:
-Z.T @ (d + G @ Y @ b)

In [ ]:
y_star = ((Z.T @ G @ Z).inv() @ (-Z.T @ (d + G @ Y @ b)))[0]
y_star

In [ ]:
x_star = Y @ b + Z @ sp.Matrix([y_star])
x_star

In [ ]:
lambda_star = -Y.T @ (G @ x_star + d)
lambda_star

## 数值求解

In [ ]:
import numpy as np
from scipy import linalg as SLA

from qp_algo import qp_eq, qp_eq_qr

In [ ]:
G_mat = np.array([[2, 5, 0], [5, 2, 0], [0, 0, 4]])
A_mat = np.array([[1, 1], [1, -2], [1, -3]])
d_vec = np.array([0, -3, -7])
b_vec = np.array([1, -2])

In [ ]:
sol, lam = qp_eq(G_mat, d_vec, A_mat, b_vec)

In [ ]:
sol

In [ ]:
lam

In [ ]:
Q_mat, R_mat = SLA.qr(A_mat)

sol, lam = qp_eq_qr(G_mat, d_vec, A_mat, Q_mat, R_mat, b_vec)

In [ ]:
sol

In [ ]:
lam

# 利用积极集法求解含不等式约束的二次规划问题

\begin{equation*}
\begin{array}{cl}
\text{minimize} & q(x) = (x_1 - 1)^2 + (x_2 - 2.5)^2, \\
\text{subject to} & -x_1 + 2x_2 - 2 \leqslant 0, \\
& x_1 + 2x_2 - 6 \leqslant 0, \\
& x_1 - 2x_2 - 2 \leqslant 0, \\
& -x_1 \leqslant 0, \\
& -x_2 \leqslant 0,
\end{array}
\end{equation*}

In [ ]:
import numpy as np
from scipy import linalg as SLA

from qp_algo import qp_active_set

In [ ]:
G_mat_asm = 2 * np.eye(2, dtype=float)
A_mat_asm = np.array([[-1, 2], [1, 2], [1, -2], [-1, 0], [0, -1]], dtype=float).T
b_vec_asm = np.array([2, 6, 2, 0, 0], dtype=float)
d_vec_asm = np.array([-2, -5], dtype=float)

In [ ]:
sol, trace_dict = qp_active_set(
    G_mat_asm,
    d_vec_asm,
    A_mat_asm,
    b_vec_asm,
    np.array([2, 0], dtype=float),
    0,
    verbose=True,
)

In [ ]:
sol